# Collocate data on avalanches and digital elevation model (DEM)

In [ ]:
url_avalanches = 'https://thredds.met.no/thredds/fileServer/arcticdata/infranor/norceavalanche/SAR-avalanche-detections-20211101-20220531.geojson'
url_dem = 'https://api.npolar.no/dataset/a660ff0c-c013-4592-a9a0-e1f3509f7fe0/_file/_all/?filename&format=zip'


In [ ]:
# Install the CVL server https://github.com/CryosphereVirtualLab/cvl-3d-viz
!pip install install git+https://github.com/CryosphereVirtualLab/cvl-3d-viz#egg=cvl-3d-viz

In [ ]:
# Clone the repository for convenience
!git clone https://github.com/CryosphereVirtualLab/cvl-3d-viz.git

In [ ]:
# Generate certificates for running the CVL server
!openssl req -x509 -nodes -days 730 -newkey rsa:2048 -keyout key.pem -out cert.pem -config cvl-3d-viz/cvl/localhost-ssl.conf

In [ ]:
# Start local CVL server
from subprocess import call
call('python cvl-3d-viz/cvl/server.py > cvlserver.log 2>&1 &', shell=True)

Visit https://localhost:3193/trust and allow insecure connection.

In [ ]:
import json
import requests

from IPython.display import IFrame
import numpy as np
from osgeo import gdal

from cvl.viz import viz, VBO

In [ ]:
# Access the avalanches data from the CVL web-page
r = requests.get(url_avalanches, allow_redirects=True)
j = json.loads(r.content)
coords = np.vstack([f['geometry']['coordinates'][0] for f in j['features']])
print(coords.shape)
print(coords[:10])

In [ ]:
# Download DEM from NPI
call(['curl', '-o', 'dem.zip', url_dem])

In [ ]:
# Unzip DEM
call(['unzip', '-o', 'dem.zip'])
call(['unzip', '-o', 'S0_DTM20_NP-ArcticDEM-Mosaic.zip'])

In [ ]:
# Read array with DEM
ds = gdal.Open('S0_DTM20_NP_ArcticDEM_Mosaic_20191216.tif')
dem = ds.ReadAsArray()

In [ ]:
# Convert coordinates of avalanche detections from UTM33 to rows and columns of DEM
# GetGeoTransform returns coefficient for computing X and Y from row and column
# InvGeoTransform computes the coefficients for the inverse task: compute row and column from input X and Y
inv_geo_trans = gdal.InvGeoTransform(ds.GetGeoTransform())
cols = (inv_geo_trans[0] + inv_geo_trans[1] * coords[:,0] + inv_geo_trans[2] * coords[:,1]).astype(int)
rows = (inv_geo_trans[3] + inv_geo_trans[4] * coords[:,0] + inv_geo_trans[5] * coords[:,1]).astype(int)

In [ ]:
# Create array for visualisation with x, y, z coordinates
height = dem[rows, cols] + 10
points = np.vstack([coords.T, height]).T
print(points.shape)
print(points[:10])

In [ ]:
# Create an object for interaction with the CVL display
visualizer = viz()

In [ ]:
# Open the CVL display
IFrame('https://nlive.norceresearch.no/cvl/', 1000, 1000)

In [ ]:
# Show the avalanche detections
vbo = VBO('points', projection=32633)
vbo.set_vertex(points[::10])

visualizer.publish_vbo('Avalanche detections', { "path" : "Avalanche" }, vbo)

In [ ]:
visualizer.look_at([15.689318737610086, 78.27205149275611, 3000], [15.639318737610086, 78.22205149275611, 0], 1)